#### Obs: Esse código vai gerar um arquivo de modelo treinado pelo Tensorflow
#### Obs: Para compilar esse código é necessário que as imagens estejam mesma pasta dele

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal
from PIL import Image


### 1º Carregamos as imagens, simplificamos os canais de cores, redimensionamos e normalizamos elas

In [ ]:
imagens_array = []
imagens_array_teste = []

for i in range (5000):
    nome_arquivo = f'/content/drive/MyDrive/Colab Notebooks/Pesquisa/meu_grafico_{i}.jpg'
    imagem = cv2.imread(nome_arquivo)   #Carregando a imagem
    imagem = cv2.resize(imagem, (200, 200)) #Redimensionamento da imagem
    matriz_imagem = np.array(imagem)
    canal_alpha = matriz_imagem[:, :, 0]   #Ficamos apenas com um canal de cor
    imagens_array.append(canal_alpha)

for i in range (1000):
    nome_arquivo = f'/content/drive/MyDrive/Colab Notebooks/Pesquisa/meu_grafico_{i+5000}.jpg'
    #imagem = Image.open(nome_arquivo)
    imagem = cv2.imread(nome_arquivo)
    imagem = cv2.resize(imagem, (200, 200))
    matriz_imagem = np.array(imagem)
    canal_alpha = matriz_imagem[:, :, 0]
    imagens_array_teste.append(canal_alpha)



In [ ]:
imagens_array=np.array(imagens_array)
print(imagens_array.shape)
imagens_array_teste=np.array(imagens_array_teste)
print(imagens_array_teste.shape)

In [ ]:
plt.figure()

plt.imshow(imagens_array_teste[1,:,:])
plt.grid(False)
plt.show()

In [ ]:
imagens_array=imagens_array/255  #Normalização dos dados
imagens_array_teste=imagens_array_teste/255

### 2º Criamos nossa arquitetura de rede neural e realizamos o treinamento

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Dense,Dropout, Conv2D, MaxPooling2D, Flatten


#Esse é a arquitetura da nossa rede neural:
model_2 = Sequential ([
    Conv2D(64, (3,3), activation='relu', input_shape=(200,200,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(512,activation='relu'),
    Dense (units=3, activation='relu')

])


from tensorflow.keras.losses import MeanSquaredError

model_2.compile (optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
loss=MeanSquaredError(),
metrics=['accuracy'])



![image.png](attachment:image.png)

In [ ]:
conjunto_treinamento=np.load("/content/drive/MyDrive/Colab Notebooks/Pesquisa/conjunto_treinamento.npy")
conjunto_teste=np.load("/content/drive/MyDrive/Colab Notebooks/Pesquisa/conjunto_teste.npy")
print(conjunto_treinamento.shape)
print(conjunto_teste.shape)

In [ ]:
'''resist_max= np.max(conjunto_treinamento[:,0])
indut_max= np.max(conjunto_treinamento[:,1])
cap_max= np.max(conjunto_treinamento[:,2])


conjunto_treinamento[:,0] = conjunto_treinamento[:,0]/resist_max
conjunto_treinamento[:,1] = conjunto_treinamento[:,1]/indut_max
conjunto_treinamento[:,2] = conjunto_treinamento[:,2]/cap_max

print("Valor máximo de resistor: ",np.max(conjunto_treinamento[:,0]))
print("Valor mínimo de resistor: ",np.min(conjunto_treinamento[:,0]))
print("Valor médio de resistor: ",np.mean(conjunto_treinamento[:,0]))
'''

In [ ]:

# Normaliza os dados
maximos1 = np.max(conjunto_treinamento, axis=0)
conjunto_treinamento_normalizado = conjunto_treinamento / maximos1
conjunto_teste_normalizado = conjunto_teste/maximos1
print(maximos1)
#print(conjunto_teste)
#print(maximos2)

In [ ]:
history=model_2.fit(imagens_array,conjunto_treinamento_normalizado,epochs=20,validation_data=(imagens_array_teste, conjunto_teste_normalizado))

### 3º Avaliamos o modelo treinado no conjunto de teste

In [ ]:
import pickle

# Salvar o objeto history em um arquivo
with open('historico_200', 'wb') as file:
    pickle.dump(history.history, file)

history_dict = history.history
history_dict.keys()
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


loss, accuracy = model_2.evaluate(imagens_array_teste, conjunto_teste)
print(loss)
print(accuracy)

In [ ]:
y_3=model_2.predict(imagens_array_teste)
np.set_printoptions(threshold=np.inf)
y_real = y_3*maximos1
print(y_3)



In [ ]:
print(y_real)

In [ ]:
print(conjunto_teste)

In [ ]:
#model_2.save('rede_treinada')

## 4º A seguir iremos ver os 30 primeiros gráficos gerados pelo conjunto original em comparação com o conjunto que o modelo previu.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  4 21:21:44 2020

@author: Edson Porto da Silva
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal

def simulador(R,L,C,r,l,c,a):

    # Resolução numérica de um circuito RLC série alimentado por uma onda quadrada
    # de tensão.
    # parâmetros do circuito:
    V0 = 0                # tensão inicial no capacitor [volts]
    I0 = 0                # corrente inicial no indutor [ampères]
    fq = 1               # frequência da onda quadrada [hertz]
    Vq = 15               # amplitude da onda quadrada [volts]
    w0 = 1/np.sqrt(L*C)   # frequência angular de ressonância [rad/segundo]

    t = np.arange(0,10,0.0001)*(2/10)   # discretização do intervalo de tempo [segundos]
    Vs = Vq*signal.square(2*np.pi*fq*t) # onda quadrada na entrada do circuito

    vC    = np.zeros(len(t))
    x     = np.zeros(len(t))

    # EDO da tensão sobre o capacitor: vc''(t)+(R/L)vc'(t)+vc(t)/LC = vs/LC

    # Solução numérica:
    vC[0] = V0       # condição incial de vc
    x[0]  = I0/C     # condição inicial da derivada vc'(t)
    #
    # Integração numérica via método de Euler:
    deltaT    = t[1]-t[0] # passo de integração
    numPoints = len(t)-1

    for kk in range(0, numPoints):
        vC[kk+1] = vC[kk]+x[kk]*deltaT                                # calcula vc(t+deltaT)
        x[kk+1]  = x[kk]+(-R/L*x[kk]-1/(L*C)*(vC[kk]-Vs[kk]))*deltaT  # calcula vc'(t+deltaT)

    # cálculo das tensões e da corrente partir de vc(t):
    i  = np.append(I0, C*np.diff(vC)/deltaT)    # corrente no circuito

    vR = R*i                                    # tensão no resistor
    vL = Vs-vR-vC                               # tensão no indutor(LKT)

    ###--------------------------------------------------------------###

    vC1    = np.zeros(len(t))
    x1     = np.zeros(len(t))

    # EDO da tensão sobre o capacitor: vc''(t)+(R/L)vc'(t)+vc(t)/LC = vs/LC

    # Solução numérica:
    vC1[0] = V0       # condição incial de vc
    x1[0]  = I0/c     # condição inicial da derivada vc'(t)
    #
    # Integração numérica via método de Euler:
    deltaT    = t[1]-t[0] # passo de integração
    numPoints = len(t)-1

    for kk in range(0, numPoints):
        vC1[kk+1] = vC1[kk]+x1[kk]*deltaT                                # calcula vc(t+deltaT)
        x1[kk+1]  = x1[kk]+(-r/l*x1[kk]-1/(l*c)*(vC1[kk]-Vs[kk]))*deltaT  # calcula vc'(t+deltaT)

    # cálculo das tensões e da corrente partir de vc(t):
    i1  = np.append(I0, c*np.diff(vC1)/deltaT)    # corrente no circuito

    vR1 = r*i1                                    # tensão no resistor
    vL1 = Vs-vR1-vC1                               # tensão no indutor(LKT)



    if a ==1:

        plt.figure(1, figsize=(8, 5), dpi=100, facecolor='w', edgecolor='k')
        plt.subplot(1, 2, 1)
        plt.plot(t, i,color="black", label='Resistência=' )
        #plt.legend(['corrente i(t)']);
        plt.grid(color='k', linestyle='--', linewidth=0.1)
        #plt.ylabel('ampères (A)', fontsize = 14)
        #plt.xlabel('tempo (s)', fontsize = 14)
        #plt.title('Corrente no circuito RLC série', fontsize = 14)

        plt.subplot(1, 2, 2)
        plt.plot(t, i1,color="black")
        plt.grid(color='k', linestyle='--', linewidth=0.1)
        plt.tight_layout()

        plt.show(block=False)



In [ ]:
for i in range (30):
    print("Figura:",i)
    simulador(conjunto_teste[i,0],conjunto_teste[i,1],conjunto_teste[i,2],y_real[i,0],y_real[i,1],y_real[i,2],1)